In [7]:
import numpy as np
import pandas as pd
import shutil
from roboflow import Roboflow
import os
import re

In [8]:
def clean_up(integral=False):
    if os.path.exists('GARBAGE-CLASSIFICATION-3-2'):
        shutil.rmtree('GARBAGE-CLASSIFICATION-3-2')
    if integral:
        if os.path.exists('data'):
            shutil.rmtree('data')
    else:
        for file in os.listdir('data'):
            file = os.path.join('data', file)
            if os.path.isfile(file):
                os.remove(file)

def move_files_recursively(src,dest='data',files=None):
    if not os.path.exists(dest):
        os.makedirs(dest)
    files_to_move = []
    if files is None:
        for root, dirs, files in os.walk(src):
            for file in files:
                if file.endswith('.jpg'):
                    files_to_move.append(os.path.join(root, file))
    else:
        interesting_files = list(set(files) & set(os.listdir(src)))
        for file in interesting_files:
            files_to_move.append(os.path.join(src, file))
    for file in files_to_move:
        shutil.move(file, dest)

In [9]:
clean_up(True)
rf = Roboflow(api_key="PYZngL70ijcagLdXpb6l")
project = rf.workspace("material-identification").project("garbage-classification-3")
dataset = project.version(2).download("tensorflow")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to GARBAGE-CLASSIFICATION-3-2 in tensorflow:: 100%|██████████| 10472/10472 [00:00<00:00, 16144.69it/s]


In [10]:
train = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/train/_annotations.csv')
test = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/test/_annotations.csv')
val = pd.read_csv('GARBAGE-CLASSIFICATION-3-2/valid/_annotations.csv')
combined_data = pd.concat([train, test, val], ignore_index=True)

In [11]:
move_files_recursively('GARBAGE-CLASSIFICATION-3-2')

In [12]:
map_classes = {
    'METAL': 'METAL_CARDBOARD_PLASTIC',
    'CARDBOARD': 'METAL_CARDBOARD_PLASTIC',
    'PLASTIC': 'METAL_CARDBOARD_PLASTIC',
    'PAPER': 'PAPER',
    'GLASS': 'GLASS',
    'BIODEGRADABLE': 'BIODEGRADABLE'
}
def get_class(c):
    for key in map_classes.keys():
        if key==c:
            return map_classes.get(key)
    return c

def remove_until_char(input_string,remove_char,include_char=True):
    match = re.search(remove_char, input_string)
    if match:
        return input_string[match.start() + 1 if include_char else 0:]
    return input_string

def get_set_name(ratios=None):
    if ratios is None:
        ratios = [0.6, 0.25, 0.15]
    rand = np.random.uniform()
    if rand <= ratios[0]:
        set_name = 'TRAIN'
    elif rand <= ratios[0] + ratios[1]:
        set_name = 'TEST'
    else:
        set_name = 'VAL'
    return set_name

def edit_row(row):
    row['class'] = get_class(row['class'])
    return row

for index, row in combined_data.iterrows():
    combined_data.loc[index] = edit_row(row)

classes_occurences = combined_data['class'].value_counts()    
classes_occurences

class
BIODEGRADABLE              45407
METAL_CARDBOARD_PLASTIC    16484
GLASS                       7809
PAPER                       4390
Name: count, dtype: int64

In [13]:
balanced_data = pd.DataFrame(columns=combined_data.columns)
for class_label in combined_data['class'].unique():
    class_samples = combined_data[combined_data['class'] == class_label].head(np.min(classes_occurences))
    balanced_data = pd.concat([balanced_data, class_samples], ignore_index=True)
print(f'Balanced dataframe shape: {balanced_data.shape}')
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)
print(f'Balanced dataframe shape shuffled: {balanced_data.shape}')

Balanced dataframe shape: (17560, 8)
Balanced dataframe shape shuffled: (17560, 8)


In [ ]:
ratios = [0.6, 0.25, 0.15]
cleansed_data = pd.DataFrame(columns=['set','filename','label','xmin_relative','ymin_relative','1','2','xmax_relative','ymax_relative','3','4'])
for index, row in balanced_data.iterrows():
    width = row['width']
    height = row['height']
    row['xmin_relative'] = row['xmin'] / width
    row['ymin_relative'] = row['ymin'] / height
    row['xmax_relative'] = row['xmax'] / width
    row['ymax_relative'] = row['ymax'] / height
    row['set'] = get_set_name(ratios)
    row['1'] = None
    row['2'] = None
    row['3'] = None
    row['4'] = None
    cleansed_data.loc[index] = row[['set','filename','class','xmin_relative','ymin_relative','1','2','xmax_relative','ymax_relative','3','4']]
    if index % 1000 == 0:
        print(f'Processed {index+1}/{balanced_data.shape[0]}')

Processed 1/17560
Processed 1001/17560
Processed 2001/17560
Processed 3001/17560


In [ ]:
cleansed_data.head().to_csv('data/annotations.csv', index=False)
cleansed_data

In [ ]:
# set,filename,class,xmin_relative,ymin_relative,_,_,xmax_relative,ymax_relative,_,_
# VAL,paper1698_jpeg.rf.8f9209f8db6c8217bb3a22c65cf69f9c.jpg,1,0,0,,,1,1,,
# TRAIN,paper305_jpg.rf.fe50060e4ff619ccfca8ece48dd09355.jpg,1,.06,.02,,,.59,.34,,
# TEST,paper1174_jpg.rf.482d3389b117a5af7a4f1a9696701419.jpg,1,.21,.13,,,.77,.35,,
# TEST,paper1230_jpg.rf.29a95d9839e17dd1e4b822fbb360074e.jpg,1,0,0,,,.74,.87,,
# TRAIN,biodegradable2166_jpeg.rf.7b649a31ebc54f2d1df322d4ebe6975d.jpg,3,.69,.53,,,.74,.58,,